# Writing out a USGSCSM ISD from a PDS3 LROCNAC image

In [1]:
import pvl
import numpy as np
import os
import pandas as pd
import knoten
import csmapi

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
from pysis import isis
from pysis.exceptions import ProcessError

## Make a CSM sensor model
imageLink must contain a URL to an LROCNAC image ending in the file name

In [2]:
imageLink = 'http://pdsimage.wr.usgs.gov/Missions/Lunar_Reconnaissance_Orbiter/LROC/EDR/LROLRC_0018/DATA/EXT/2013354/NAC/M1142142198RE.IMG'
!wget -N -P data {imageLink}
fileName = 'data/' + os.path.split(imageLink)[1]
camera = knoten.csm.create_csm(fileName)

--2019-07-16 12:55:22--  http://pdsimage.wr.usgs.gov/Missions/Lunar_Reconnaissance_Orbiter/LROC/EDR/LROLRC_0018/DATA/EXT/2013354/NAC/M1142142198RE.IMG
Resolving pdsimage.wr.usgs.gov (pdsimage.wr.usgs.gov)... 8.27.177.76, 2001:1900:2200:19ff::76
Connecting to pdsimage.wr.usgs.gov (pdsimage.wr.usgs.gov)|8.27.177.76|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pdsimage2.wr.usgs.gov/Missions/Lunar_Reconnaissance_Orbiter/LROC/EDR/LROLRC_0018/DATA/EXT/2013354/NAC/M1142142198RE.IMG [following]
--2019-07-16 12:55:22--  https://pdsimage2.wr.usgs.gov/Missions/Lunar_Reconnaissance_Orbiter/LROC/EDR/LROLRC_0018/DATA/EXT/2013354/NAC/M1142142198RE.IMG
Resolving pdsimage2.wr.usgs.gov (pdsimage2.wr.usgs.gov)... 8.27.177.76, 2001:1900:2200:19ff::76
Connecting to pdsimage2.wr.usgs.gov (pdsimage2.wr.usgs.gov)|8.27.177.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 259281864 (247M) [application/octet-stream]
Server file no newer than lo

## Ingest the image and spiceinit

In [3]:
# Set the output location of the resulting .cub
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.lronac2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

## Define a function that compares ISIS and USGSCSM pixels

In [4]:
def check_pixel(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using ISIS, then projects
    the result back into an image coordinate using USGSCSM and computes the difference
    between image coordinates.
    """
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to subtract (.5,.5) from the ISIS pixels
    line_diff = pvl_output['GroundPoint']['Line'] - image_coord.line - .5
    sample_diff = pvl_output['GroundPoint']['Sample'] - image_coord.samp - .5
    return line_diff, sample_diff

## Get the total number of lines / samples

In [5]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

## Compare top left, top right, bottom left, bottom right, and center pixels using check_pixel

In [6]:
pixels_dict = {'line' : [1,1,n_lines, n_lines, n_lines/2],
               'sample' : [1, n_samples, 1, n_samples, n_samples/2]}

pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff'] = np.NaN
pixels_df['sample_diff'] = np.NaN

for idx, row in pixels_df.iterrows():
    pixels_df.iloc[idx]['line_diff'], pixels_df.iloc[idx]['sample_diff'] = check_pixel(camera, cub_loc, row['line'], row['sample'])

pixels_df

,line,sample,line_diff,sample_diff
0,1.0,1.0,23.101415,-18.007311
1,1.0,5064.0,11.372646,39.146329
2,51200.0,1.0,-207.084592,-19.375048
3,51200.0,5064.0,-219.021746,37.766173
4,25600.0,2532.0,-97.874974,9.989890
